In [1]:
import tensorflow as tf 
from tensorflow import keras
import pandas as pd
import tensorflow_hub as hub

from tensorflow.keras.preprocessing.image  import ImageDataGenerator
import os
import datetime
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from tensorboard import notebook


In [2]:
#tf.keras.backend.clear_session()


In [3]:
tf.__version__

'2.0.0'

# Tensorboard

In [4]:
#%load_ext tensorboard


In [5]:
#notebook.list()

In [6]:
#%tensorboard --logdir logs


In [7]:
DATASET_PATH="../data/myntradataset"

In [8]:
DATASET_PATH_IMAGES="%s/images".format(DATASET_PATH)


# Clean Data

In [9]:
df = pd.read_csv(DATASET_PATH+"/styles.csv",  error_bad_lines=False)

b'Skipping line 6044: expected 10 fields, saw 11\nSkipping line 6569: expected 10 fields, saw 11\nSkipping line 7399: expected 10 fields, saw 11\nSkipping line 7939: expected 10 fields, saw 11\nSkipping line 9026: expected 10 fields, saw 11\nSkipping line 10264: expected 10 fields, saw 11\nSkipping line 10427: expected 10 fields, saw 11\nSkipping line 10905: expected 10 fields, saw 11\nSkipping line 11373: expected 10 fields, saw 11\nSkipping line 11945: expected 10 fields, saw 11\nSkipping line 14112: expected 10 fields, saw 11\nSkipping line 14532: expected 10 fields, saw 11\nSkipping line 15076: expected 10 fields, saw 12\nSkipping line 29906: expected 10 fields, saw 11\nSkipping line 31625: expected 10 fields, saw 11\nSkipping line 33020: expected 10 fields, saw 11\nSkipping line 35748: expected 10 fields, saw 11\nSkipping line 35962: expected 10 fields, saw 11\nSkipping line 37770: expected 10 fields, saw 11\nSkipping line 38105: expected 10 fields, saw 11\nSkipping line 38275: ex

In [10]:
df['image'] = df.apply(lambda row: str(row['id']) + ".jpg", axis=1)

In [11]:
df.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,39386.jpg
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,59263.jpg
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,21379.jpg
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,53759.jpg


In [12]:
df_counts = df['subCategory'].value_counts()
df_counts

Topwear                     15402
Shoes                        7343
Bags                         3055
Bottomwear                   2694
Watches                      2542
Innerwear                    1808
Jewellery                    1079
Eyewear                      1073
Fragrance                    1011
Sandal                        963
Wallets                       933
Flip Flops                    913
Belts                         811
Socks                         698
Lips                          527
Dress                         478
Loungewear and Nightwear      470
Saree                         427
Nails                         329
Makeup                        307
Headwear                      293
Ties                          258
Accessories                   129
Scarves                       118
Cufflinks                     108
Apparel Set                   106
Free Gifts                    104
Stoles                         90
Skin Care                      77
Skin          

only consider categories that have 300 images or so

In [13]:
min_images = 300
top_classes = df_counts[df_counts>min_images].index.tolist()
top_classes

['Topwear',
 'Shoes',
 'Bags',
 'Bottomwear',
 'Watches',
 'Innerwear',
 'Jewellery',
 'Eyewear',
 'Fragrance',
 'Sandal',
 'Wallets',
 'Flip Flops',
 'Belts',
 'Socks',
 'Lips',
 'Dress',
 'Loungewear and Nightwear',
 'Saree',
 'Nails',
 'Makeup']

In [14]:
df = df[df['subCategory'].isin(top_classes)]
df

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,39386.jpg
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,59263.jpg
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,21379.jpg
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,53759.jpg
...,...,...,...,...,...,...,...,...,...,...,...
44419,17036,Men,Footwear,Shoes,Casual Shoes,White,Summer,2013.0,Casual,Gas Men Caddy Casual Shoe,17036.jpg
44420,6461,Men,Footwear,Flip Flops,Flip Flops,Red,Summer,2011.0,Casual,Lotto Men's Soccer Track Flip Flop,6461.jpg
44421,18842,Men,Apparel,Topwear,Tshirts,Blue,Fall,2011.0,Casual,Puma Men Graphic Stellar Blue Tshirt,18842.jpg
44422,46694,Women,Personal Care,Fragrance,Perfume and Body Mist,Blue,Spring,2017.0,Casual,Rasasi Women Blue Lady Perfume,46694.jpg


In [15]:
len(top_classes)

20

In [16]:
df = df.groupby('subCategory').apply(pd.DataFrame.sample, n=min_images).reset_index(drop=True)
df

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,25239,Women,Accessories,Bags,Handbags,Olive,Summer,2012.0,Casual,Lino Perros Women Classic Olive Handbag,25239.jpg
1,22649,Unisex,Accessories,Bags,Backpacks,Green,Winter,2015.0,Casual,Wildcraft Unisex Green & Grey Backpack,22649.jpg
2,15660,Unisex,Accessories,Bags,Handbags,Black,Fall,2011.0,Casual,Belkin Unisex Dash Laptop 16,15660.jpg
3,52766,Women,Accessories,Bags,Handbags,Black,Fall,2012.0,Casual,Fiorelli Women Black Leather Handbag,52766.jpg
4,37699,Unisex,Accessories,Bags,Backpacks,Black,Spring,2013.0,Casual,American Tourister Unisex Black Backpack,37699.jpg
...,...,...,...,...,...,...,...,...,...,...,...
5995,25327,Men,Accessories,Watches,Watches,White,Winter,2016.0,Casual,CASIO Edifice Men White Dial Watch ED424,25327.jpg
5996,54650,Women,Accessories,Watches,Watches,Red,Winter,2016.0,Casual,Fastrack Women White Dial Watch,54650.jpg
5997,45933,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Maxima Women Silver Dial Watch,45933.jpg
5998,47965,Men,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Helix Men Sliver Dial Watch,47965.jpg


shuffle the dataframe

In [17]:
df = df.sample(frac=1).reset_index(drop=True)


In [18]:
len(df)

6000

In [19]:
df.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,54152,Women,Accessories,Jewellery,Necklace and Chains,Gold,Fall,2012.0,Casual,Femella Golden Necklace,54152.jpg
1,49944,Men,Apparel,Loungewear and Nightwear,Lounge Shorts,Black,Summer,2016.0,Casual,Hanes Men Black Shorts,49944.jpg
2,28791,Men,Apparel,Bottomwear,Jeans,Black,Summer,2012.0,Casual,Denizen Men Black Jeans,28791.jpg
3,52478,Women,Apparel,Loungewear and Nightwear,Baby Dolls,Red,Winter,2015.0,Casual,Red Rose Red Nightdress,52478.jpg
4,2366,Girls,Accessories,Bags,Backpacks,Black,Winter,2010.0,Sports,ADIDAS Black Girls Backpack,2366.jpg


In [20]:
batch_size = 32

# Data Generators

create iterators across our dataset

In [21]:
IMAGE_SHAPE = (224, 224)


In [22]:
image_generator = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest" 
    
)

training_generator = image_generator.flow_from_dataframe(
    dataframe=df,
    directory=DATASET_PATH + "/images",
    x_col="image",
    y_col="subCategory",
    target_size=IMAGE_SHAPE,
    batch_size=batch_size,
    subset="training"
)

validation_generator = image_generator.flow_from_dataframe(
    dataframe=df,
    directory=DATASET_PATH + "/images",
    x_col="image",
    y_col="subCategory",
    target_size=IMAGE_SHAPE,
    batch_size=batch_size,
    subset="validation"
)

classes = len(training_generator.class_indices)

Found 4800 validated image filenames belonging to 20 classes.
Found 1199 validated image filenames belonging to 20 classes.


/usr/local/lib/python3.5/dist-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 1 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  .format(n_invalid, x_col)


# Model

In [23]:
""" Old Way
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(weights='imagenet', 
                      include_top=False, 
                      input_shape = IMAGE_SHAPE+(3,))
base_model.trainable = False


model = tf.keras.Sequential([
    base_model
    , tf.keras.layers.GlobalAveragePooling2D()

    ,tf.keras.layers.Dense(classes, activation='softmax')
])

"""

" Old Way\nbase_model = tf.keras.applications.mobilenet_v2.MobileNetV2(weights='imagenet', \n                      include_top=False, \n                      input_shape = IMAGE_SHAPE+(3,))\nbase_model.trainable = False\n\n\nmodel = tf.keras.Sequential([\n    base_model\n    , tf.keras.layers.GlobalAveragePooling2D()\n\n    ,tf.keras.layers.Dense(classes, activation='softmax')\n])\n\n"

In [25]:
# Add base layer
classifier_url ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4" 
base_model = hub.KerasLayer(classifier_url,  input_shape = IMAGE_SHAPE+(3,), trainable=False)



model = tf.keras.Sequential([
    base_model
    ,tf.keras.layers.Dense(classes, activation='softmax') # classification layer
])

In [26]:
model.build()

In [27]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 1001)              3540265   
_________________________________________________________________
dense_1 (Dense)              (None, 20)                20040     
Total params: 3,560,305
Trainable params: 20,040
Non-trainable params: 3,540,265
_________________________________________________________________


fit the model

In [28]:
#?model.fit_generator
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

#callbacks = [tensorboard_callback]
callbacks = []

In [29]:
model.fit_generator(
    generator=training_generator,
    validation_data=validation_generator,
    epochs=3,
    verbose=1,
    workers=8,
    callbacks=callbacks
    #use_multiprocessing=True
)

Epoch 1/3
150/150 [==============================] - 107s 711ms/step - loss: 1.0584 - accuracy: 0.6704 - val_loss: 0.6351 - val_accuracy: 0.7965
Epoch 2/3
150/150 [==============================] - 91s 603ms/step - loss: 0.4852 - accuracy: 0.8383 - val_loss: 0.5146 - val_accuracy: 0.8290
Epoch 3/3
150/150 [==============================] - 91s 608ms/step - loss: 0.4076 - accuracy: 0.8598 - val_loss: 0.4984 - val_accuracy: 0.8299


# Saving the model

In [ ]:
# Save the model
model.save('model_fashion.h5')

# Recreate the exact same model purely from the file
new_model = keras.models.load_model('model_fashion.h5',custom_objects={'KerasLayer':hub.KerasLayer})

Export the model to a SavedModel

In [ ]:
#!rm -rf 'model.pb'
#model.save('model.pb', save_format='tf')
#new_model = keras.models.load_model('model.pb')

## INCOMPPLETE

# Accuracy

In [30]:
# get list of class names
#target_names = list(validation_generator.class_indices.keys())

# map of id-> true label
label_dict = dict((v,k) for k,v in validation_generator.class_indices.items())

# get most confidant class
probabilities = model.predict_generator(validation_generator)
y_pred = np.argmax(probabilities,axis=1)


df_result = pd.DataFrame({'file_name': validation_generator.filenames,
    'y_true': validation_generator.classes,
    'y_pred': y_pred})


df_result['y_true'] = df_result['y_true'].apply(lambda x: label_dict[x])
df_result['y_pred'] = df_result['y_pred'].apply(lambda x: label_dict[x])


In [31]:
df_result[df_result['y_true']!=df_result['y_pred']]

,file_name,y_pred,y_true
0,54152.jpg,Innerwear,Jewellery
1,49944.jpg,Shoes,Loungewear and Nightwear
2,28791.jpg,Saree,Bottomwear
3,52478.jpg,Bottomwear,Loungewear and Nightwear
4,2366.jpg,Dress,Bags
...,...,...,...
1194,55212.jpg,Belts,Lips
1195,18463.jpg,Watches,Fragrance
1196,48466.jpg,Innerwear,Dress
1197,43816.jpg,Nails,Jewellery


In [33]:
?confusion_matrix

In [32]:
cm = confusion_matrix(, y_true=y_true, y_pred=y_pred)

cm

SyntaxError: invalid syntax (<ipython-input-32-911f14a6aba5>, line 1)

In [ ]:
sns.heatmap(cm, xticklabels=target_names, annot=True , yticklabels=target_names )# font size
